In [2]:
import gym
from env_batch import ParallelEnvBatch


def make_env():
    return gym.make('Pendulum-v0')


env = ParallelEnvBatch(make_env, nenvs=16)

print(env.observation_space.shape)
print(env.action_space.shape)

(3,)
(1,)


In [3]:
curr = env.reset()
print(curr.shape)

import numpy as np

obs = np.array([curr for _ in range(10)])

(16, 3)


In [1]:
from rlkits.policies import PolicyWithValue
from rlkits.env_batch import ParallelEnvBatch
import numpy as np
import gym
from rlkits.sampler import TrajectorySampler

def make_env():
    return gym.make('Pendulum-v0')

env = ParallelEnvBatch(make_env, nenvs=16)

#env = make_env()


ob_space = env.observation_space
ac_space = env.action_space

pi = PolicyWithValue(
    ob_space=ob_space, ac_space=ac_space, ckpt_dir='/tmp', 
    hidden_layers=[1024])


samp = TrajectorySampler(env, pi, 5)
#ac, log_prob, v = policy.step(curr)

Not a Vector Env
<class 'rlkits.env_batch.ParallelEnvBatch'>


In [5]:
if not isinstance(env, ParallelEnvBatch):
    print('false')

In [18]:
nx = np.random.rand(16, 3)

t = np.random.rand(16)
done = (t > 0.5)

In [21]:
done.shape

(16,)

In [20]:
nx[done]

array([[0.92563383, 0.55998717, 0.65613426],
       [0.94821461, 0.88353904, 0.36528434],
       [0.15110719, 0.7119068 , 0.7472019 ],
       [0.75858051, 0.70755973, 0.4250092 ],
       [0.69849213, 0.30402498, 0.11032107],
       [0.14048728, 0.29181518, 0.38692375],
       [0.79454275, 0.70728973, 0.29947912]])

In [ ]:
nx_state (2, 3)
rew (2,)
done (2,) bool

In [30]:
nxstate = np.array([
    [ 0.97061545, -0.24063592, -0.565105],
 [-0.18096242,0.98349001,2.07499883]])
done = np.array([False,False])

nxstate[~done]

array([[ 0.97061545, -0.24063592, -0.565105  ],
       [-0.18096242,  0.98349001,  2.07499883]])

In [24]:
nx_state = np.random.rand(2, 3)
done = np.array([False, False])

nx_state[~done]

array([[0.76228153, 0.38908259, 0.09040862],
       [0.76929873, 0.14689109, 0.05569494]])

# how to compute average episode rewards for a parallel env?
```
curr_ep_ret = [e1, e2, ..., en]

if env 2 is done
ep_ret[2].append(curr_ep_ret[2])
curr_ep_ret[2] = 0

```
so I get a list of list for `ep_rets` such that `ep_rets[i]` is the return from env i

Same idea can be applied to episodic length. Create a list of list for the 
`ep_lens` variable
such that `ep_lens[i]` is the length of episode for env i. 